In [ ]:
%%capture
!pip install transformers==2.8.0

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
import pandas as pd
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel, GPT2Config
import torch

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from nltk.tokenize import word_tokenize
import numpy as np
import random


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
rouge = Rouge()

print ("module %s loaded" % module_url)
def encode(input):
  return model(input)

def use_score_pairs(left, right):
    return np.inner(encode(left), encode(right))

def plot_similarity(text, paratext, rotation=90):
    corr = score_pairs(text, paratext)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
        corr,
        xticklabels=paratext,
        yticklabels=text,
        vmin=0,
        vmax=1,
        cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")

def bleu_score_pairs(left, right):
    left = map(word_tokenize, map(str.strip, map(str.lower, left)))
    right = map(word_tokenize, map(str.strip, map(str.lower, right)))
    return [sentence_bleu([x],y) for x,y in zip(left, right)]

def rouge_score_pairs(left, right):
    left = list(map(str.strip, map(str.lower, left)))
    right =  list(map(str.strip, map(str.lower, right)))
    rouge_scores =  rouge.get_scores(left, right)
    return [x['rouge-l']['f'] for x in rouge_scores]

ModuleNotFoundError: ignored

In [ ]:
folder = "/gdrive/MyDrive/diploma/models/gpt2-medium_quora_mpc_subs_v1_endoftext_v2.8.0_2021-03-14_09:16:07/epoch_1_iter_8000/"
#folder = "/gdrive/MyDrive/diploma/models/gpt2-medium_toxic_to_non-toxic_v1_endoftext_v2.8.0_2021-05-17_22:18:43/epoch_1_iter_7051/"



model = GPT2LMHeadModel.from_pretrained(folder)
tokenizer = GPT2Tokenizer.from_pretrained(folder)
# model.to(device)

In [ ]:
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [ ]:
import numpy as np
import random

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
People are fucking morons.

SyntaxError: ignored

In [ ]:
prompt = "Get the fuck out of this page.:paraphrase:"
#prompt = "<|startoftext|> From the 18th through the late 20th century, all the history of science, especially of the physical and biological sciences, was often presented as a progressive accumulation of knowledge. <|sep|>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to("cuda")

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: Get the fuck out of this page.:paraphrase:You got this page off the Internet. If you want it back, please return it.


1: Get the fuck out of this page.:paraphrase:How do you get rid of clutter in your home?


2: Get the fuck out of this page.:paraphrase:Open this page.


3: Get the fuck out of this page.:paraphrase:Get out of here.


4: Get the fuck out of this page.:paraphrase:Take this page out of your browser.


5: Get the fuck out of this page.:paraphrase:Get out of here.


6: Get the fuck out of this page.:paraphrase:Put this page out of order.


7: Get the fuck out of this page.:paraphrase:How do I get out of here?


8: Get the fuck out of this page.:paraphrase:Put the page down.


9: Get the fuck out of this page.:paraphrase:You're not allowed to go.




In [ ]:
device="cuda"

In [ ]:
device='cpu'

In [ ]:
def generate_paraphrase_greedy(text):
    try:
        generated = torch.tensor(tokenizer.encode(text+":paraphrase:")).unsqueeze(0).to(device)
        gen = model.generate(generated, 
                                    do_sample=True,   
                                    top_k=40, 
                                    max_length = 300,
                                    top_p=0.95, 
                                    num_return_sequences=10
                                    )
        return [tokenizer.decode(x, skip_special_tokens=True).split(":paraphrase:")[1] for x in gen]
    except UnboundLocalError as e:
        print(e)
        return ""

In [ ]:
generate_paraphrase_greedy("Leave me alone asshole.")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


["I'm alone in the world.",
 'I have to go alone.',
 'What are some ways to make a friend and get famous in no time?',
 "My whole life, I was selfish and insecure. What's one thing you want from me?",
 'Leave me alone, baby.',
 'Shall I come with you?',
 'Leave me alone, darling.',
 'You gonna call me on that?',
 'I have no one but you to blame for this.',
 'I should get some other place.']

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import re
import json

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
df = pd.read_csv("/gdrive/MyDrive/diploma/data/prepared_datasets/mpc.csv", index_col=0)

In [ ]:


#df = pd.read_csv("/gdrive/MyDrive/diploma/data/toxicity/jigsaw_toxic_6k.csv")

df["paraphrase"] = df['sentence1'].progress_map(lambda x: json.dumps(generate_paraphrase_greedy(x)))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [ ]:
df[df['paraphrase'].map(bool)].to_csv("/gdrive/MyDrive/diploma/data/metrics/output_data/mpc_paraphrase_GPT2.csv")

In [ ]:
df

,Unnamed: 0,comment_text,paraphrase
0,16,"Bye! Don't look, come or think of comming back...","[""Come back, not thinking of what you just did..."
1,55,Stupid peace of shit stop deleting my stuff as...,"[""Damn, we got to the point where this app is ..."
2,79,Hi! I am back again! Last warning! Stop undoin...,"[""I am back again. Stop editing. Please stop d..."
3,86,"Would you both shut up, you don't run wikipedi...","[""Both shut up shut up shut up shut up shut up..."
4,105,A pair of jew-hating weiner nazi schmucks.,"[""Two of the men, Florin Cioaba and Vladimiro ..."
...,...,...,...
6274,152141,She's a little self obssessed and self righteo...,"[""She's a little self-indulgent and self-loath..."
6275,152327,I will meet you one day and stab you to death ...,"[""On Monday, I will meet you one day and stab ..."
6276,152414,Oh that's not an airport...that's just where t...,"[""That's not an airport.\n"", ""That's not an ai..."
6277,152477,That is because you are a retard. Business str...,"[""It is because you are a retard.\n"", ""You are..."


In [ ]:
prompt = "<|startoftext|> The term knowledge can refer to a theoretical or practical understanding of a subject.<|sep|>"
#prompt = "<|startoftext|> From the 18th through the late 20th century, all the history of science, especially of the physical and biological sciences, was often presented as a progressive accumulation of knowledge. <|sep|>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  The term knowledge can refer to a theoretical or practical understanding of a subject.The terms knowledge are used to describe the qualities of a subject and what it might convey in a future.


1:  The term knowledge can refer to a theoretical or practical understanding of a subject.The word knowledge is basically about a domain - what is known, and what is not.


2:  The term knowledge can refer to a theoretical or practical understanding of a subject.The term knowledge means a formalism about a subject and an interest.


